In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import gc
import re
import glob

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
from tqdm.auto import tqdm 

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, auc

from urllib.parse import quote, unquote, urlparse

import lightgbm as lgb

Matplotlib created a temporary cache directory at C:\Users\Jupiter\AppData\Local\Temp\matplotlib-wjuq87t6 because the default path (C:\Users\Jupiter\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
import os
print(os.getcwd())
os.chdir(r'C:\Users\Jupiter\Desktop\dataming\web_attack_and_classification\CCF-BDCI2022-Web-Attack-Detection-and-Classification-master')
print(os.getcwd())

D:\jupyternotebook_dewenjian\lgb-baseline
C:\Users\Jupiter\Desktop\dataming\web_attack_and_classification\CCF-BDCI2022-Web-Attack-Detection-and-Classification-master


In [ ]:
import random
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(42)

In [ ]:

train_files = glob.glob('./data/train/*.csv')

df_train = pd.DataFrame()

for filepath in tqdm(train_files):
    df = pd.read_csv(filepath)
    df_train = pd.concat([df_train, df]).reset_index(drop=True)
df_train.fillna('__NaN__', inplace=True)

df_train = df_train.rename(columns={'lable': 'label'})
df_train.head()


In [ ]:
df_test = pd.read_csv('./data/test/test.csv')
df_test.fillna('__NaN__', inplace=True)
df_test

In [3]:
df=pd.read_csv('./df.csv')

In [ ]:
df = pd.concat([df_train, df_test]).reset_index(drop=True)
df.shape

In [ ]:
def get_url_query(s):
    li = re.split('[=&]', urlparse(s)[4])
    return [li[i] for i in range(len(li)) if i % 2 == 1]


def find_max_str_length(x):
    max_ = 0
    li = [len(i) for i in x]
    return max(li) if len(li) > 0 else 0


def find_str_length_std(x):
    max_ = 0
    li = [len(i) for i in x]
    return np.std(li) if len(li) > 0 else -1

df['url_unquote'] = df['url'].apply(unquote)


In [ ]:
df['url_length'] = df['url'].str.len()
df['url_special_chars'] = df['url'].apply(lambda x: len(re.findall(r'[^a-zA-Z0-9]', x)))

dangerous_keywords = ['select', 'union', 'delete', 'drop', 'update', 'exec', 'script']
df['url_dangerous_keywords'] = df['url_unquote'].apply(
        lambda x: sum(1 for keyword in dangerous_keywords if keyword.lower() in x.lower())
    )

df['url_params_count'] = df['url'].apply(lambda x: len(urlparse(x).query.split('&')))

In [ ]:
def find_url_filetype(x):
    try:
        return re.search(r'\.[a-z]+', x).group()
    except:
        return '__NaN__'
df['url_path'] = df['url_unquote'].apply(lambda x: urlparse(x)[2])
df['url_filetype'] = df['url_path'].apply(lambda x: find_url_filetype(x))
df['url_path_len'] = df['url_path'].apply(len)
df['url_path_num'] = df['url_path'].apply(lambda x: len(re.findall('/',  x)))
# find_url_filetype("https://example/image.jpg")

In [ ]:
df['ua_short'] = df['user_agent'].apply(lambda x: x.split('/')[0])
df['ua_first'] = df['user_agent'].apply(lambda x: x.split(' ')[0])
df['ua_os'] = df['user_agent'].apply(lambda x: re.search(r'\((.*?)\)', x).group(1) if re.search(r'\((.*?)\)', x) else '')
# 浏览器版本提取
df['ua_browser_version'] = df['user_agent'].apply(
    lambda x: re.search(r'Version/(\d+)', x).group(1) if re.search(r'Version/(\d+)', x) else -1
    )
# UA长度
df['ua_length'] = df['user_agent'].str.len()

In [ ]:
 # Body长度
df['body_length'] = df['body'].str.len()
    # JSON检测
df['is_json'] = df['body'].apply(lambda x: 1 if x.strip().startswith('{') else 0)
    # 特殊字符比例
df['body_special_char_ratio'] = df['body'].apply(
    lambda x: len(re.findall(r'[^a-zA-Z0-9\s]', x)) / (len(x) + 1)
    )

In [ ]:
# %%time

def add_tfidf_feats(df, col, n_components=16):
    text = list(df[col].values)
    tf = TfidfVectorizer(min_df=3, 
                         max_df=0.95,
                         analyzer='char_wb',
                         ngram_range=(1,6),
                         stop_words='english',
                         max_features=10000)
    tf.fit(text)
    X = tf.transform(text)
    svd = TruncatedSVD(n_components=n_components)
    svd.fit(X)
    X_svd = svd.transform(X)
    explained_variance_ratio=svd.explained_variance_ratio_
    for i in range(n_components):
        df[f'{col}_tfidf_{i}'] = X_svd[:, i]*explained_variance_ratio[i]
    return df

df = add_tfidf_feats(df, 'url', n_components=64)
df = add_tfidf_feats(df, 'user_agent', n_components=32)
df = add_tfidf_feats(df, 'body', n_components=64)
df = add_tfidf_feats(df, 'refer', n_components=16)

In [ ]:
df['method_filetype'] = df['method'].astype(str) + '_' + df['url_filetype'].astype(str)
df['ua_method'] = df['ua_short'].astype(str) + '_' + df['method'].astype(str)


for col in tqdm(['method', 'url_filetype', 'ua_short', 'ua_first','ua_os','method_filetype','ua_method']):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

df['url_param_length_ratio'] = df['url_path_len'] / (df['url_params_count'] + 1)
    # UA类型与请求方法组合


In [4]:
not_use_feats = ['id', 'user_agent', 'url', 'body', 'url_unquote', 'url_query', 'url_path', 'label' ,'refer','method'
                  #'url_query_num', 'url_query_max_len', 'url_query_len_std','ua_short', 'ua_first',
#                  'url_filetype','url_path_len','url_path_num','url_unquote_tfidf_6','url_unquote_tfidf_7','user_agent_tfidf_2','user_agent_tfidf_4','user_agent_tfidf_7','user_agent_tfidf_8','user_agent_tfidf_9','user_agent_tfidf_13','user_agent_tfidf_15','body_tfidf_4','body_tfidf_5','body_tfidf_6','body_tfidf_7','body_tfidf_13','body_tfidf_15'
]
not_use_feats2 = ['id', 'user_agent', 'url', 'body', 'url_unquote', 'url_query', 'url_path']
use_features = [col for col in df.columns if col not in not_use_feats]

In [7]:
train = df[df['label'].notna()]
test = df[df['label'].isna()]

train.head()


,id,method,user_agent,url,refer,body,label,Nums_XSS_attack_feat_body,Nums_SQL_attack_feat_body,Nums_special_char_feat_body,Nums_special_char_feat_url,url_unquote,url_query,url_query_num,url_query_max_len,url_query_len_std,url_length,url_special_chars,url_params_count,url_path,url_filetype,url_path_len,url_path_num,ua_short,ua_first,ua_os,ua_length,url_unquote_tfidf_0,url_unquote_tfidf_1,url_unquote_tfidf_2,url_unquote_tfidf_3,url_unquote_tfidf_4,url_unquote_tfidf_5,url_unquote_tfidf_6,url_unquote_tfidf_7,url_unquote_tfidf_8,url_unquote_tfidf_9,url_unquote_tfidf_10,url_unquote_tfidf_11,url_unquote_tfidf_12,url_unquote_tfidf_13,url_unquote_tfidf_14,url_unquote_tfidf_15,url_unquote_tfidf_16,url_unquote_tfidf_17,url_unquote_tfidf_18,url_unquote_tfidf_19,url_unquote_tfidf_20,url_unquote_tfidf_21,url_unquote_tfidf_22,url_unquote_tfidf_23,url_unquote_tfidf_24,url_unquote_tfidf_25,url_unquote_tfidf_26,url_unquote_tfidf_27,url_unquote_tfidf_28,url_unquote_tfidf_29,url_unquote_tfidf_30,url_unquote_tfidf_31,url_unquote_tfidf_32,url_unquote_tfidf_33,url_unquote_tfidf_34,url_unquote_tfidf_35,url_unquote_tfidf_36,url_unquote_tfidf_37,url_unquote_tfidf_38,url_unquote_tfidf_39,url_unquote_tfidf_40,url_unquote_tfidf_41,url_unquote_tfidf_42,url_unquote_tfidf_43,url_unquote_tfidf_44,url_unquote_tfidf_45,url_unquote_tfidf_46,url_unquote_tfidf_47,url_unquote_tfidf_48,url_unquote_tfidf_49,url_unquote_tfidf_50,url_unquote_tfidf_51,url_unquote_tfidf_52,url_unquote_tfidf_53,url_unquote_tfidf_54,url_unquote_tfidf_55,url_unquote_tfidf_56,url_unquote_tfidf_57,url_unquote_tfidf_58,url_unquote_tfidf_59,url_unquote_tfidf_60,url_unquote_tfidf_61,url_unquote_tfidf_62,url_unquote_tfidf_63,url_unquote_tfidf_64,url_unquote_tfidf_65,url_unquote_tfidf_66,url_unquote_tfidf_67,url_unquote_tfidf_68,url_unquote_tfidf_69,url_unquote_tfidf_70,url_unquote_tfidf_71,url_unquote_tfidf_72,url_unquote_tfidf_73,url_unquote_tfidf_74,url_unquote_tfidf_75,url_unquote_tfidf_76,url_unquote_tfidf_77,url_unquote_tfidf_78,url_unquote_tfidf_79,url_unquote_tfidf_80,url_unquote_tfidf_81,url_unquote_tfidf_82,url_unquote_tfidf_83,url_unquote_tfidf_84,url_unquote_tfidf_85,url_unquote_tfidf_86,url_unquote_tfidf_87,url_unquote_tfidf_88,url_unquote_tfidf_89,url_unquote_tfidf_90,url_unquote_tfidf_91,url_unquote_tfidf_92,url_unquote_tfidf_93,url_unquote_tfidf_94,url_unquote_tfidf_95,url_unquote_tfidf_96,url_unquote_tfidf_97,url_unquote_tfidf_98,url_unquote_tfidf_99,url_unquote_tfidf_100,url_unquote_tfidf_101,url_unquote_tfidf_102,url_unquote_tfidf_103,url_unquote_tfidf_104,url_unquote_tfidf_105,url_unquote_tfidf_106,url_unquote_tfidf_107,url_unquote_tfidf_108,url_unquote_tfidf_109,url_unquote_tfidf_110,url_unquote_tfidf_111,url_unquote_tfidf_112,url_unquote_tfidf_113,url_unquote_tfidf_114,url_unquote_tfidf_115,url_unquote_tfidf_116,url_unquote_tfidf_117,url_unquote_tfidf_118,url_unquote_tfidf_119,url_unquote_tfidf_120,url_unquote_tfidf_121,url_unquote_tfidf_122,url_unquote_tfidf_123,url_unquote_tfidf_124,url_unquote_tfidf_125,url_unquote_tfidf_126,url_unquote_tfidf_127,user_agent_tfidf_0,user_agent_tfidf_1,user_agent_tfidf_2,user_agent_tfidf_3,user_agent_tfidf_4,user_agent_tfidf_5,user_agent_tfidf_6,user_agent_tfidf_7,user_agent_tfidf_8,user_agent_tfidf_9,user_agent_tfidf_10,user_agent_tfidf_11,user_agent_tfidf_12,user_agent_tfidf_13,user_agent_tfidf_14,user_agent_tfidf_15,user_agent_tfidf_16,user_agent_tfidf_17,user_agent_tfidf_18,user_agent_tfidf_19,user_agent_tfidf_20,user_agent_tfidf_21,user_agent_tfidf_22,user_agent_tfidf_23,user_agent_tfidf_24,user_agent_tfidf_25,user_agent_tfidf_26,user_agent_tfidf_27,user_agent_tfidf_28,user_agent_tfidf_29,user_agent_tfidf_30,user_agent_tfidf_31,user_agent_tfidf_32,user_agent_tfidf_33,user_agent_tfidf_34,user_agent_tfidf_35,user_agent_tfidf_36,user_agent_tfidf_37,user_agent_tfidf_38,user_agent_tfidf_39,user_agent_tfidf_40,user_agent_tfidf_41,user_agent_tfidf_42,user_agent_tfidf_43,user_agent_tfidf_44,user_agent_tfidf_45,user_agent_tfidf_46,user_agent_tfi

In [8]:
from lightgbm import log_evaluation, early_stopping
from sklearn.utils.class_weight import compute_class_weight


In [9]:
NUM_CLASSES = 6
FOLDS = 10
TARGET = 'label'

from sklearn.preprocessing import label_binarize

def run_lgb(df_train, df_test, use_features):
    
    target = TARGET
    oof_pred = np.zeros((len(df_train), NUM_CLASSES))
    y_pred = np.zeros((len(df_test), NUM_CLASSES))
    
     # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(df_train[target]), y=df_train[target])
    class_weight_dict = {i: class_weights[i] for i in range(NUM_CLASSES)}
    custom_weights={0: 0.204/0.422,1: 0.201/0.299,2: 0.199/0.195,3: 0.176/0.042,4: 0.116/0.020,5: 0.101/0.019}
    
    folds = StratifiedKFold(n_splits=FOLDS)
    for fold, (tr_ind, val_ind) in enumerate(folds.split(train, train[TARGET])):
        print(f'Fold {fold + 1}')
        x_train, x_val = df_train[use_features].iloc[tr_ind], df_train[use_features].iloc[val_ind]
        y_train, y_val = df_train[target].iloc[tr_ind], df_train[target].iloc[val_ind]
        
        
        
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)
        
        params = {
            'learning_rate': 0.05,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': NUM_CLASSES,
            'feature_fraction': 0.6,
            'bagging_fraction': 0.9,
            'min_data_in_leaf':100,
            'bagging_freq': 10,
            'n_jobs': -1,
            'nthread': -1,
            'max_depth': 8,
            'num_leaves': 31,
            'lambda_l1': 0.5,
            'lambda_l2': 0.8,
            'verbose': -1,
        }
        
        model = lgb.train(params, 
                          train_set, 
                          num_boost_round=500,
                          callbacks = [log_evaluation(100), early_stopping(stopping_rounds=60)],
                          valid_sets=[train_set, val_set],
                          )
        oof_pred[val_ind] = model.predict(x_val)
        y_pred += model.predict(df_test[use_features]) / folds.n_splits
        
        print("Features importance...")
        gain = model.feature_importance('gain')
        feat_imp = pd.DataFrame({'feature': model.feature_name(), 
                         'split': model.feature_importance('split'), 
                         'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
        # print('Top 50 features:\n', feat_imp.head(50))
        
        del x_train, x_val, y_train, y_val, train_set, val_set
        gc.collect()
        
    return y_pred, oof_pred
y_pred, oof_pred = run_lgb(train, test, use_features)
print(accuracy_score(np.argmax(oof_pred, axis=1), df_train['label']))

Fold 1
Training until validation scores don't improve for 60 rounds
[100]	training's multi_logloss: 0.015356	valid_1's multi_logloss: 0.0515269
[200]	training's multi_logloss: 0.0049558	valid_1's multi_logloss: 0.0502169
Early stopping, best iteration is:
[169]	training's multi_logloss: 0.00609376	valid_1's multi_logloss: 0.0489936
Features importance...
Fold 2
Training until validation scores don't improve for 60 rounds
[100]	training's multi_logloss: 0.0147047	valid_1's multi_logloss: 0.063929
Early stopping, best iteration is:
[135]	training's multi_logloss: 0.00837674	valid_1's multi_logloss: 0.0623789
Features importance...
Fold 3
Training until validation scores don't improve for 60 rounds
[100]	training's multi_logloss: 0.0152051	valid_1's multi_logloss: 0.049241
Early stopping, best iteration is:
[133]	training's multi_logloss: 0.0089481	valid_1's multi_logloss: 0.0472077
Features importance...
Fold 4
Training until validation scores don't improve for 60 rounds
[100]	training's

NameError: name 'df_train' is not defined

In [10]:
sub = pd.read_csv('./data/submit_example.csv')
sub['predict'] = np.argmax(y_pred, axis=1)
sub['predict'].value_counts()

2    886
1    815
0    809
3    654
4    436
5    400
Name: predict, dtype: int64

In [11]:
sub.to_csv('baseline.csv', index=False)